In [ ]:
pip install datasets -q

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/IDEFICS2/

/content/drive/MyDrive/Colab Notebooks/IDEFICS2


In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import json
from PIL import Image
import os

# Function to load a JSON file
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Function to save a JSON file
def save_json(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

# Function to convert img_name to PIL image object
def load_images(data, root_dir):
    for entry in data:
        img_path = os.path.join(root_dir, entry['img_name'])
        entry['img'] = Image.open(img_path)
    return data

# Load dataset
train_data = load_json('datasets/SLAKE1/train.json')
test_data = load_json('datasets/SLAKE1/test.json')
validate_data = load_json('datasets/SLAKE1/validation.json')

# Directory containing the images
root_dir = 'datasets/SLAKE1/imgs'

# Convert image paths to PIL image objects
train_data_addimage = load_images(train_data, root_dir)
test_data_addimage = load_images(test_data, root_dir)
validate_data_addimage = load_images(validate_data, root_dir)

In [ ]:
# Function to filter entries based on language
def filter_by_language(data, language):
    return [entry for entry in data if entry.get('q_lang') == language]

# Filter entries by language
train_data_en = filter_by_language(train_data_addimage, 'en')
test_data_en = filter_by_language(test_data_addimage, 'en')
validate_data_en = filter_by_language(validate_data_addimage, 'en')

In [ ]:
# because train data idx 1622 is detected to be "None"
train_data_en[1622]['answer'] = "No"

In [ ]:
train_data_en[1622]

{'img_id': 281,
 'img_name': 'xmlab281/source.jpg',
 'question': 'Does the picture contain liver?',
 'answer': 'No',
 'q_lang': 'en',
 'location': 'Chest_lung',
 'modality': 'CT',
 'answer_type': 'OPEN',
 'base_type': 'vqa',
 'content_type': 'Organ',
 'triple': ['vhead', '_', '_'],
 'qid': 1622,
 'img': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>}

In [ ]:
from datasets import Dataset

# # Create a new dataset with only the necessary features
newtrain_temp = {
    'image': [entry['img'] for entry in train_data_en],
    'question': [entry['question'] for entry in train_data_en],
    'answer': [entry['answer'] for entry in train_data_en]
}

newval_temp = {
    'image': [entry['img'] for entry in validate_data_en],
    'question': [entry['question'] for entry in validate_data_en],
    'answer': [entry['answer'] for entry in validate_data_en]
}

newtest_temp = {
    'image': [entry['img'] for entry in test_data_en],
    'question': [entry['question'] for entry in test_data_en],
    'answer': [entry['answer'] for entry in test_data_en]
}

# Convert the dictionary to a Dataset object
newtrain_data = Dataset.from_dict(newtrain_temp)
newval_data = Dataset.from_dict(newval_temp)
newtest_data = Dataset.from_dict(newtest_temp)

In [ ]:
from datasets import DatasetDict
dataset = DatasetDict({
    "train": newtrain_data,
    "validation": newval_data,
    "test": newtest_data,
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'question', 'answer'],
        num_rows: 4919
    })
    validation: Dataset({
        features: ['image', 'question', 'answer'],
        num_rows: 1053
    })
    test: Dataset({
        features: ['image', 'question', 'answer'],
        num_rows: 1061
    })
})

In [ ]:
dataset.push_to_hub("mdwiratathya/SLAKE-vqa-english", commit_message="Fix empty answer on qid 1622")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/2460 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Map:   0%|          | 0/2459 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1061 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/590 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mdwiratathya/SLAKE-vqa-english/commit/08a290fb60e94cbf80650496048cb8d10160c883', commit_message='Fix empty answer on qid 1622', commit_description='', oid='08a290fb60e94cbf80650496048cb8d10160c883', pr_url=None, pr_revision=None, pr_num=None)